<a href="https://colab.research.google.com/github/avasiliki/MyProjects/blob/main/search_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Εξάγωγή ειδήσεων από RSS feeds 5 ειδησεογραφικών καναλιών: enikos, naftemporiki, in,protothema και tanea.Στη συνέχεια αποθήκευση αυτών στο dataframe df.

In [ ]:
!pip install feedparser
from dateutil import parser
import feedparser
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
import pytz
import re

In [ ]:
timeZ_At = pytz.timezone('Europe/Athens') #παίρνουμε τη ζώνη ώρας για την Ελλάδα
# Εξαγωγή ειδήσεων από το ειδησεογραφικό κανάλι enikos.gr
def read_enikos_rss():

 #URL προς απόξεση
 url_to_scrape="https://www.enikos.gr/feed/"
 NewsFeed = feedparser.parse(url_to_scrape)
 
 news_items=[]  
 for entry in NewsFeed.entries:
    news_item = {}
    news_item['title'] = entry.title
    news_item['category'] = entry.category
    news_item['description'] = entry.description
    news_item['link'] = entry.link
    datetime_rss = parser.parse(entry.published)
    datetime_timestamp = float(datetime_rss.astimezone(timeZ_At).strftime("%s"))    #μετατροπή ώρας στην τοπική
    local_datetime_converted = datetime.datetime.fromtimestamp(datetime_timestamp)  #μετατροπή ημερομηνίας/ώρας σε μορφή κοινή για όλα τα ειδησεογραφικά κανάλια
    news_item['pubDate'] = local_datetime_converted 

    #  Ανάγνωση του περιεχομένου των ειδήσεων 
    news = requests.get(entry.link)
    news_content = news.content  
    soup_news = BeautifulSoup(news_content, 'html.parser')
    for script in soup_news(['script']):
         script.decompose()  # αφαίρεση ετικετών script και του περιεχομένου τους
        
    b = soup_news.find_all('div', class_='articletext') 
    try:
       x=b[0].get_text()
    except IndexError:
       x = ""

    # Αφαίρεση κενών γραμμών από το περιεχόμενο
    lines = x.split("\n")
    non_empty_lines = [line for line in lines if line.strip() != ""]
    text_without_empty_lines = ""
    for line in non_empty_lines:
      text_without_empty_lines += line + " "
            
    news_item['content']= re.sub(r"http\S+", "", text_without_empty_lines)
    
    news_items.append(news_item)
 return news_items

# Εξαγωγή ειδήσεων από το ειδησεογραφικό κανάλι naftemporiki.gr
def read_naftemporiki_rss():

 #URL προς απόξεση
 url_to_scrape="https://www.naftemporiki.gr/rssFeed"
 NewsFeed = feedparser.parse(url_to_scrape)

 news_items=[]  
 for entry in NewsFeed.entries:
    news_item = {}
    news_item['title'] = entry.title
    news_item['category'] = ' '
    news_item['description'] = entry.description
    news_item['link'] = entry.link
    datetime_rss = parser.parse(entry.published)
    datetime_timestamp = float(datetime_rss.strftime("%s"))
    local_datetime_converted = datetime.datetime.fromtimestamp(datetime_timestamp) #μετατροπή ημερομηνίας/ώρας σε μορφή κοινή για όλα τα ειδησεογραφικά κανάλια
    news_item['pubDate'] = local_datetime_converted 

    #  Ανάγνωση του περιεχομένου των ειδήσεων 
    news = requests.get(entry.link)
    news_content = news.content   
    soup_news = BeautifulSoup(news_content, 'html.parser')
    b = soup_news.find_all('span', id='spBody')
    try:
       x = b[0].find_all(['p','li'])
    except IndexError:
       x = []

    # Ένωση των παραγράφων
    list_paragraphs = []
    final_article=" "
    for p in range(len(x)):
        paragraph = x[p].get_text() + " "
        list_paragraphs.append(paragraph) 
    final_article = " ".join(list_paragraphs)
    news_item['content']=final_article 
    
    news_items.append(news_item)
 return news_items

# Εξαγωγή ειδήσεων από το ειδησεογραφικό κανάλι in.gr 
def read_in_rss():

 #URL προς απόξεση
 url_to_scrape="https://www.in.gr/feed/"
 NewsFeed = feedparser.parse(url_to_scrape)

 news_items=[]  
 for entry in NewsFeed.entries:
    news_item = {}
    news_item['title'] = entry.title
    news_item['category'] = entry.category
    soup_description= BeautifulSoup(entry.description, 'html.parser')
    news_item['description'] = soup_description.get_text()  
    news_item['link'] = entry.link
    datetime_rss = parser.parse(entry.published)
    datetime_timestamp = float(datetime_rss.astimezone(timeZ_At).strftime("%s"))   #μετατροπή ώρας στην τοπική
    local_datetime_converted = datetime.datetime.fromtimestamp(datetime_timestamp) #μετατροπή ημερομηνίας/ώρας σε μορφή κοινή για όλα τα ειδησεογραφικά κανάλια
    news_item['pubDate'] = local_datetime_converted 

    #  Ανάγνωση του περιεχομένου των ειδήσεων 
    news = requests.get(entry.link)
    
    news_content = news.content   
    soup_news = BeautifulSoup(news_content, 'html.parser')
    #ανάγνωση ειδήσεων από το h2 tag στο πάνω  μέρος του άρθρου
    b1=soup_news.find_all('h2', class_='description' )
    #ανάγνωση του υπόλοιπου άρθρου ειδήσεων
    b = soup_news.find_all('div', itemprop='articleBody')

    # το άρθρο δεν έχει σταθερή μορφή    
    try:
      if len(b)==2:  # 1η μορφή
       x = b[1].find_all(["p","h1","h2","h3","h4"])
      else: # 2η μορφή
       x = b[0].find_all(["p","h1","h2","h3","h4"])
    except IndexError:
      x = []
     
           
    # Ένωση των παραγράφων
    list_paragraphs = []
    final_article="  "
    try:
      paragraph=b1[0].get_text() #κείμενο απο το h2 tag στο πάνω  μέρος του άρθρου     
    except:
      paragraph=""     
    list_paragraphs.append(paragraph) 

    if len(b)==2: # 1η μορφή
      for p in range(len(x)):
        paragraph = x[p].get_text() + " "
        list_paragraphs.append(paragraph) 
    else:  #2η μορφή
     try:
       paragraph = x[0].get_text() + " " 
     except:
       paragraph=""
     list_paragraphs.append(paragraph) 
    final_article = " ".join(list_paragraphs) #τελικό περιεχόμενο του άρθρου
    news_item['content']=final_article 
        
    news_items.append(news_item)
 return news_items




# Εξαγωγή ειδήσεων από το ειδησεογραφικό κανάλι protothema.gr 
def read_protothema_rss():

 #URL προς απόξεση
 url_to_scrape="https://www.protothema.gr/rss"
 NewsFeed = feedparser.parse(url_to_scrape)
 

 news_items=[]  
 for entry in NewsFeed.entries:
    news_item = {}
    news_item['title'] = entry.title
    news_item['category'] = entry.category
    soup_description= BeautifulSoup(entry.description, 'html.parser')
    news_item['description'] = soup_description.get_text()  
    news_item['link'] = entry.link
    datetime_rss = parser.parse(entry.published)
    datetime_timestamp = float(datetime_rss.strftime("%s"))
    local_datetime_converted = datetime.datetime.fromtimestamp(datetime_timestamp)  #μετατροπή ημερομηνίας/ώρας σε μορφή κοινή για όλα τα ειδησεογραφικά κανάλια
    news_item['pubDate'] = local_datetime_converted 

    #  Ανάγνωση του περιεχομένου των ειδήσεων 
    news = requests.get(entry.link)
    news_content = news.content   
    soup_news = BeautifulSoup(news_content, 'html.parser')
    for script in soup_news(['script']):
         script.decompose()    # αφαίρεση ετικετών script και του περιεχομένου τους
        
       
    #ανάγνωση ειδήσεων από το h3 tag στο πάνω  μέρος του άρθρου
    try:
      div_element = soup_news.find('div', class_='content details')
      children = div_element.findChildren('h3' , recursive=False)
      h3=children[0].get_text()
    except:
      h3=""
      
     
    #ανάγνωση του υπόλοιπου άρθρου ειδήσεων
    b = soup_news.find_all('div', class_='cntTxt')   
    try:
      x = b[0].get_text()
    except IndexError:
      x = ""
    # Αφαίρεση κενών γραμμών από το περιεχόμενο
    lines = x.split("\n")
    non_empty_lines = [line for line in lines if line.strip() != ""]
    text_without_empty_lines = h3 + " "
    for line in non_empty_lines:
      if line.find("Ειδήσεις σήμερα")!=-1 or line=="Ακολουθήστε το protothema.gr στο Google News και μάθετε πρώτοι όλες τις ειδήσεις" :
        break
      if line.find("Glomex Player")!=-1: 
        continue  
      text_without_empty_lines += line.strip() + " "
    news_item['content']=text_without_empty_lines.replace('\n', ' ').strip()
    news_items.append(news_item)
 
 return news_items


# Εξαγωγή ειδήσεων από το ειδησεογραφικό κανάλι tanea.gr 
def read_tanea_rss():

 #URL προς απόξεση
 url_to_scrape="https://www.tanea.gr/rss"
 NewsFeed = feedparser.parse(url_to_scrape)

 news_items=[]  
 for entry in NewsFeed.entries:
    news_item = {}
    news_item['title'] = entry.title
    news_item['category'] = entry.category
    soup_description= BeautifulSoup(entry.description, 'html.parser')
    news_item['description'] = soup_description.get_text()
    news_item['link'] = entry.link
    datetime_rss = parser.parse(entry.published)
    datetime_timestamp = float(datetime_rss.astimezone(timeZ_At).strftime("%s"))     #μετατροπή ώρας στην τοπική
    local_datetime_converted = datetime.datetime.fromtimestamp(datetime_timestamp)   #μετατροπή ημερομηνίας/ώρας σε μορφή κοινή για όλα τα ειδησεογραφικά κανάλια
    news_item['pubDate'] = local_datetime_converted 
    soup_content= BeautifulSoup(entry.content[0]['value'], 'html.parser')             #εξαγωγή του περιεχομένου του άρθρου της είδησης
    news_item['content'] =news_item['description'] + soup_content.get_text().replace('\n', ' ')
    news_items.append(news_item)
 return news_items

#επανάληψη της διαδικασίας της εξαγωγής των ειδήσεων κάθε μία ώρα
while 1:
  #Δημιουργία dataframe df1 από τις ειδήσεις που έχουν εξαχθεί από το κανάλι enikos.gr 
 df = pd.DataFrame(read_enikos_rss(), columns=['title','category', 'description','link','content','pubDate'])
 try:
    df1=df1.append(df) #συνένωση με το dataframe df1 της προηγούμενης επανάληψης
 except:
    df1=df 
 df1=df1.drop_duplicates(subset ="title") #αφαίρεση διπλότυπων ειδήσεων
 
 #Δημιουργία dataframe df2 από τις ειδήσεις που έχουν εξαχθεί από το κανάλι naftemporiki.gr  
 df = pd.DataFrame(read_naftemporiki_rss(), columns=['title','category', 'description','link','content','pubDate'])
 try:
    df2=df2.append(df) #συνένωση με το dataframe df2 της προηγούμενης επανάληψης
 except:
    df2=df 
 df2=df2.drop_duplicates(subset ="title") #αφαίρεση διπλότυπων ειδήσεων
 
 #Δημιουργία dataframe df3 από τις ειδήσεις που έχουν εξαχθεί από το κανάλι in.gr  
 df = pd.DataFrame(read_in_rss(), columns=['title','category', 'description','link','content','pubDate'])
 try:
    df3=df3.append(df)  #συνένωση με το dataframe df3 της προηγούμενης επανάληψης
 except:
    df3=df 
 df3=df3.drop_duplicates(subset ="title")  #αφαίρεση διπλότυπων ειδήσεων
 
 #Δημιουργία dataframe df4 από τις ειδήσεις που έχουν εξαχθεί από το κανάλι protothema.gr  
 df = pd.DataFrame(read_protothema_rss(), columns=['title','category', 'description','link','content','pubDate'])
 try:
    df4=df4.append(df)  #συνένωση με το dataframe df4 της προηγούμενης επανάληψης
 except:
    df4=df 
 df4=df4.drop_duplicates(subset ="title") #αφαίρεση διπλότυπων ειδήσεων
 
 #Δημιουργία dataframe df5 από τις ειδήσεις που έχουν εξαχθεί από το κανάλι tanea.gr  
 df = pd.DataFrame(read_tanea_rss(), columns=['title','category', 'description','link','content','pubDate'])
 try:
    df5=df5.append(df)  #συνένωση με το dataframe df5 της προηγούμενης επανάληψης
 except:
    df5=df 
 df5=df5.drop_duplicates(subset ="title")  #αφαίρεση διπλότυπων ειδήσεων 
  
 #συνένωση των dataframes df1, df2, df3, df4, df5
 frames = [df1, df2, df3,df4,df5]
 df = pd.concat(frames)
 df.reset_index(drop=True, inplace=True)

 display(df)
 time.sleep(60*60) #επανάληψη της διαδικασίας εξαγωγής των ειδήσεων κάθε μία ώρα

# Προεπεξεργασία ειδήσεων

In [ ]:
import re
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import unicodedata as ud
!pip install greek-stemmer
from greek_stemmer import GreekStemmer

In [4]:
stemmer = GreekStemmer()  # stemmer για την ελληνική γλώσσα

stopWordsGreek = stopwords.words('greek')      # Λήψη της λίστας με τις ελληνικές στοπ λέξεις 
stopWordsEnglish = stopwords.words('english')  # Λήψη της λίστας με τις αγγλικές στοπ λέξεις 
stopWords=stopWordsGreek + stopWordsEnglish

# Προσθήκη νέων stop λέξεων στη λίστα
stopWords.extend(['ακόμη','ακόμα','ακριβώς','αλλά','άλλος', 'αλλού','άλλα','άλλη','άλλο','άλλοι','άλλων','άλλους','άλλες','άμεσα','αμέσως','αν','ανά','ανάμεσα','αναφέρει','ανέφερε','αντ', 'αντί','αντίθετα','αντίστοιχα','άνω', 'από','απ','άρα','αργά','αργότερα','αρκετά','αρκετός','αρκετή','αρκετοί','αρκετές','αρκετό','αρκετών','αρκετούς','αρχικά','αρχικό','αρχική','αρχικός','αρχικοί','αρχικές','αρχικούς','αρχικών','ας','αφορά','αφού','αφότου','αύριο', 'αυτά', 'αυτές', 'αυτή','αυτήν', 'αυτό', 'αυτοί', 'αυτός', 'αυτούς', 'αυτών','αυτόν','αυτής','αυτού','βέβαια','γενικά','γενικώς','γρ','για','γιατί','γι','γίνομαι','γίνεται','γρήγορα','γύρω','διαβάστε','δηλαδή','δήλωσε','δε','δει','δείτε','δειτε','δεν','δικό','δικός','δική','δικοί','δικούς','δικών','διότι','δίπλα','δύο','δυο',
                   'εάν','εαυτόν','εαυτό','έγκαιρα','εγκαίρως','εδώ','εδω','είδα','είδες','είδε','είδαμε','είδατε','είδαν','είδανε','ειδικά','είπα','είπες','είπε','είπαμε','είπατε','είπαν','εκ','έκανε','έκαναν','έγινε','έγιναν','εν','εντός','έναντι','ένα','έναν','ένας','ενός','ενόψει','επί','επι','επιπλέον','εκεί', 'εγώ','εσύ','είμαι', 'είμαστε', 'είναι', 'είσαι', 'είστε', 'εκείνα', 'εκείνες', 'εκείνη','εκείνο', 'εκείνος', 'εκείνοι', 'εκείνους', 'εκείνων','εμάς','εσάς','εμείς','εσείς','εξάλλου','εξής','έξω','έπειτα','επίσης','επιπλέον','επειδή','επομένως','έπρεπε', 'ενώ','εμένα','εσένα','εσυ','έτσι', 'έως','είπε', 'είπαμε','είπατε','είπαν',
                   'είτε','εκτός','εμπρός','εντελώς','έχω','έχεις','έχει','έχουμε','έχετε','έχουν','είχε','είχαν','είχα','είχαμε','είχατε','είχες','ευρέως','εφόσον','ήμουν','ήσουν','ήταν','η','ή', 'ήδη','θα','ίδια','ίδιος','ίδιο','ίδιες','ίδιοι', 'ιδίως','ίσα','ίσια','ίσως', 'και', 'κάνω','κάνεις','κάνει','κάνουμε','κάνετε','κάνουν','κάθε', 'καθένας','καθεμια','καθεμία','καθόλου','καθώς','καλά','κακά', 'καμία','καμια','κατά','κι','καν','κανείς','κανένας','κανένα','κάποιος','κάποια','κάποιο','κάποιοι','κάποιες','κάποτε','κάτι','κάτω','κιόλας','κοντά','κυρίως','λέει','λένε','λιγάκι','λίγο','λίγες','λίγων','λίγοι','λίγα','λίγος','λίγης','λίγους','λιγότερο',
                   'λιγότερα','λιγότερος','λιγότεροι','λιγότερη','λιγότερες','λιγότερων','λοιπόν','λόγω','μαζί','μάλιστα','μάλλον','μέσα','μέσω','μεταξύ','μετά','μέχρι','μη','μήπως','μίλησε','μόλις','μας','μένα' ,'με','μερικοί','μερικών','μερικούς','μερικές','μερικά','μολονότι','μην','μόνο','μόνον','μόνος','μόνη','μου','μια','μία','μιας','μίας','μπορώ','μπορείς','μπορεί','μπορούμε','μπορείτε','μπορούν','νά','να','ναι','νομίζω','νομίζεις','νομίζει','νομίζουμε','νομίζετε','νομίζουν','νωρίς','νωρίτερα','ξανά', 'ο', 'οι','όλο','όλος','όλη','όλοι','όλες','όλα','όλους','όλων','όμως','όντως','όπου','οποία','οποίο','οποίος','οποίων','οποίους','οποίοι','οποίου','οποίες',
                   'οπότε','όπως','ορισμένο','ορισμένα','ορισμένοι','ορισμένες','όσο','όσος','όση','όσα','όσες','όσοι','όσων','όσους','όσον','όποιος','όποιο','όποια','όποιο','όποιων','όποιοι','όποιους','όποιες','οποιοσδήποτε','οποιαδήποτε','οποιοδήποτε','οποιουδήποτε','οποιασδήποτε','οποιοιδήποτε','οποιεσδήποτε','οποιωνδήποτε','οποιουσδήποτε','ος', 'ότι', 'ό,τι','όταν','ούτε','όχι','πάντα','πάλι','πάνω','πάρα','παρακολουθήστε','παρόλα','παρόλο','πει','πέρα','περίπου','περισσότερα','πια','πίσω','πιθανό','πιθανόν','ποια','ποιες', 'ποιο', 'ποιοι','ποιόν','ποιος','ποιον', 'πότε','ποτέ','πιο','πλέον','ποιους', 'ποιων','πόσο','πόσος','πόση','πόσα','πόσοι','πόσες','που','πού',
                   'πολύ','πολλοί','πολλά','πολλών','πολλούς','πολλές','πράγματι','πρέπει','πριν','πρόκειται','προτού','πως','πώς','σαν','σας','σε','σου','σιγά','σήμερα', 'στη', 'στην', 'στο', 'στον', 'στα','στιγμή','στις','στους','συ','σύμφωνα','συνεπώς','συνέχεια','συνεχώς','συνήθως','συχνά','σχεδόν','σχετικός','σχετική','σχετικό','σωστά','τα','ταυτόχρονα','ταυτόχρονη','τελικά','το', 'την', 'τη','τι', 'τις','της','των','τέτοιο','τέτοια','τέτοιος','τέτοιοι','τέτοιων','τέτοιους','τέτοιες','τέτοιον','τίποτα','τόνισε','τόνισαν','τουλάχιστον','τόσο','τόσα','τόσος','τόση','τόσοι','τόσων','τόσους','τόσες','τούτος','τούτη','τούτο','τούτα','τούτες','τον',
                   'τότε', 'του', 'τους','τώρα','υπάρχει','υπάρξει','υπάρχουν','υπό','ύστερα','ως','ώσπου','ώστε','ωστόσο','προς','μία','φαίνεται','φορά','φυσικά','χαμηλά','χάρη','χθες','χωρίς', 'proto', 'thema','Πηγή:www.gazzetta.gr','Πηγή: ΑΠΕ-ΜΠΕ','marieclaire','gr','www','gazzetta','com','πηγή','απε','μπε','pic','twitter','live','view','post','instagram','shared','ygeiamou','epiruspostgr','https'])

d = {ord('\N{COMBINING ACUTE ACCENT}'):None} # για την αφαίρεση τόνων από τις ελληνικές λεξεις

In [5]:
# συνάρτηση για δημιουργία tokens και καθαρισμός του κειμένου
def tokenize_only(str_input):
    bagOfWords = nltk.word_tokenize(str_input) # δημιουργία tokens
    
    clean_tokens=[]
   
    for token in bagOfWords:
      token = token.lower()
      # αφαίρεση κάθε τιμής που δεν ανήκει στο ελληνικό ή αγγλικό αλφάβητο
      new_token = re.sub(r'[^α-ωΑ-Ωά-ώΆ-ΏΆ-Ώa-zA-Z]+', '', token) 
      # αφαίρεση των tokens μεγέθους ενός  χαράακτήρα
      if  len(new_token.strip()) >= 2: 
         vowels=len([v for v in new_token if v in "αάεέιίηήοόυύωώaeiouy"])
         if vowels != 0 and new_token not in stopWords: # αφαίρεση των λέξεων που περιέχουν μόνο σύμφωνα και δεν ανήκουν στη λίστα των stop λέξεων
             clean_tokens.append(new_token)     
          
    return clean_tokens




In [6]:
# συνάρτηση για δημιουργία tokens,καθαρισμός του κειμένου και stemming
def tokenize_and_stem(str_input):
    bagOfWords = nltk.word_tokenize(str_input) # δημιουργία tokens
    
    stem_sentence=[]
    for token in bagOfWords:
      token = token.lower()
      # αφαίρεση κάθε τιμής που δεν ανήκει στο ελληνικό ή αγγλικό αλφάβητο
      new_token = re.sub(r'[^α-ωΑ-Ωά-ώΆ-ΏΆ-Ώa-zA-Z]+', '', token) 
      # αφαίρεση των tokens μεγέθους ενός  χαράακτήρα
      if  len(new_token.strip()) >= 2: 
         vowels=len([v for v in new_token if v in "αάεέιίηήοόυύωώaeiouy"])
         if vowels != 0 and new_token not in stopWords:  # αφαίρεση των λέξεων που περιέχουν μόνο σύμφωνα και δεν ανήκουν στη λίστα των stop λέξεων
               stem_sentence.append(stemmer.stem(ud.normalize('NFD',new_token.upper()).translate(d)))  # μετατροπή σε κεφαλαία, αφαίρεση τόνων και stemming των λέξεων
  
    return stem_sentence

In [ ]:
# δημιουργία dataframe vocab_frame όπου κάθε γραμμή περιέχει την λέξη που έχει υποστεί stemming και την λέξη χωρίς stemming
totalvocab_stemmed = []     # λίστα με όλα τα tokens που έχουν υποστεί stemming
totalvocab_tokenized = []   # λίστα με όλα τα tokens
for i in df['content'].tolist():
    allwords_stemmed = tokenize_and_stem(i) # για κάθε είδηση του dataframe, tokenize και stemming
    totalvocab_stemmed.extend(allwords_stemmed) # επέκταση της 'totalvocab_stemmed' λίστας
    
    allwords_tokenized = tokenize_only(i)  # για κάθε είδηση του dataframe, tokenize
    totalvocab_tokenized.extend(allwords_tokenized)# επέκταση της 'totalvocab_tokenized' λίστας

vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed) # δημιουργία του dataframe vocab_frame
display(vocab_frame.head())




# Ομαδοποίηση ειδήσεων με k-means

In [8]:

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

Μετατροπή της συλλογής των ειδήσεων σε έναν tf-idf matrix

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words=stopWords, tokenizer=tokenize_and_stem,
                       analyzer='word',lowercase=True)
                      
dtm_tfidf = tfidf_vectorizer.fit_transform(df['content'].tolist())

Εφαρμογή αλγορίθμου k-means για ομαδοποίηση ειδήσεων

In [ ]:
num_clusters=int(input('Give the number of clusters: ')) # Πλήθος των ομάδων ειδήσεων που θα δημιουργηθούν
# Έκτέλεση του αλγορίθμου k-means
km = KMeans(n_clusters=num_clusters)
km.fit(dtm_tfidf)
 
clusters = km.labels_.tolist()
news = { 'title': df['title'].tolist(), 'category': df['category'].tolist(), 'content': df['content'].tolist(), 'cluster': clusters }
frame = pd.DataFrame(news,  columns = ['cluster', 'category', 'title', 'content']) # Δημιουργία dataframe με στήλες:Cluster, κατηγορία, τίτλος, περιεχόμενο 
print("Number of of news articles per cluster")
print(frame['cluster'].value_counts()) # εμφάνιση πλήθους ειδήσεων ανά cluster

Στατιστικά και γραφήματα από την ομαδοποίηση ειδήσεων

In [ ]:


prediction = km.predict(dtm_tfidf) # προβλέπει την πλησιέστερη ομάδα που ανήκει κάθε μια από τις ειδήσεις
# συνάρτηση για στατιστικά και γραφήματα από την ομαδοποίηση ειδήσεων
def get_top_words_cluster(tf_idf_array, prediction, n_w):
    labels = np.unique(prediction)
        
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indexes  για κάθε cluster
        x_means = np.mean(tf_idf_array[id_temp], axis = 0) # επιστρέφει τη μέση βαθμολογία στο cluster
        sorted_means = np.argsort(x_means)[::-1][:n_w] # indexes με τις top 15 βαθμολογίες
        features = tfidf_vectorizer.get_feature_names_out()
        top_words=[]
        # εμφάνιση ολόκληρων των λέξεων και όχι των αποκομμένων
        for i in sorted_means:
           if len(vocab_frame.loc[features[i],'words'][0])==1:
             t=(vocab_frame.loc[features[i],'words'], x_means[i])
           else:
             t=(vocab_frame.loc[features[i],'words'][0], x_means[i])
           top_words.append(t) 
        df = pd.DataFrame(top_words, columns = ['words', 'score']) # δημιουργία dataframe με στήλες τις λέξεις και τη βαθμολογία
        df.plot.barh(x ='words', y='score', color=['red','blue'])	# γραφική αναπαράσταση του dataframe
        plt.title(label)
        plt.show()
        print(df)
        dfs.append(df)
get_top_words_cluster(dtm_tfidf.toarray(), prediction, 15)


# LDA Visualization

In [ ]:
from __future__ import print_function

!pip install pyldavis==2.1.2
import pyLDAvis
import pyLDAvis.sklearn
import warnings
warnings.filterwarnings("ignore")

from sklearn.decomposition import LatentDirichletAllocation

Μετατροπή της συλλογής των ειδήσεων σε έναν tf matrix

In [13]:
tf_vectorizer = CountVectorizer(analyzer='word', stop_words=stopWords, 
                             lowercase=True,tokenizer=tokenize_and_stem)
dtm_tf = tf_vectorizer.fit_transform(df['content'].tolist())
 

Δημιουργία lda μοντέλου και οπτικοποίηση των topics

In [ ]:
# topic modelling - 20 topics
lda_tf = LatentDirichletAllocation(n_components=20,max_iter=20, random_state=20)
lda_tf.fit(dtm_tf)
# οπτικοποίηση των topics
pyLDAvis.enable_notebook()
vis=pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer) 
display(pyLDAvis.display(vis))

 


# Faiss search

In [ ]:
!pip install faiss-gpu
!pip install -U sentence-transformers


import numpy as np
import torch
import os
import pandas as pd
import faiss
import time
from sentence_transformers import SentenceTransformer

In [ ]:
# λήψη του μοντέλου 'sn-xlm-roberta-base-snli-mnli-anli-xnli' και δημιουργία embeddings
model = SentenceTransformer('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli')
data=df.content.to_list()
encoded_data = model.encode(data)

In [ ]:
# Αποθήκευση των sentence embeddings σε έναν faiss index
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(data))))

In [ ]:
#Αλλαγή από cpu σε gpu
res = faiss.StandardGpuResources()
gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

# Εγκατάσταση ElasticSearch

Εγκατάσταση elasticsearch server version 7.12.0. 

In [ ]:
!apt install default-jdk > /dev/null
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.12.0-linux-x86_64.tar.gz -q --show-progress
!tar -xzf elasticsearch-7.12.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.12.0


Εκκίνηση του Elasticsearch Server

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.12.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
# Αναμονή για λίγα δευτερόλεπτα μέχρι να ξεκινήσει ο server.
import time
time.sleep(20)

Εγκατάσταση του Elasticsearch client και έλεγχος αν ο client μπορεί να έχει πρόσβαση στον Elasticsearch server.Εκτέλεση δύο φορές αν χρειάζεται, ώστε να δώσουμε χρόνο στον server να ξεκινήσει.

In [ ]:
!pip install elasticsearch==7.12.0 -q
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")
es.ping()  # πρέπει να λάβουμε True

Έλεγχος αν το elasticsearch δουλεύει κανονικά 

In [ ]:
!curl -X GET "localhost:9200/"

# Προσθήκη των ειδήσεων στον ElasticSearch index

In [ ]:
# Σύνδεση στο elasticsearch 
es = Elasticsearch("http://localhost:9200")
#διαγραφή του  index αν υπάρχει ήδη
es.indices.delete(index='news_index', ignore=[400, 404])
# δημιουργία του index ανα δεν υπάρχει
try:
    request_body = {
        "settings" : {
            "number_of_shards": 1,
            "number_of_replicas": 1
        },

        "mappings": {
            "properties": {
                "title": {"type": "text"},
                "category": {"type": "text"},
                "description": {"type": "text"},
                "link": {"type":  "text"},
                "pubDate": {"type": "text"},
                "body": {"type": "text"},
                "body_vector": {"type":  "dense_vector", "dims":  768}
            }
        }
    }
    es.indices.create(index = 'news_index', body = request_body)
except:
    pass

# Προσθήκη κάθε άρθρου είδήσεων στο index. 
for i in range(len(df)):
            news = {
                'title': df['title'][i],
                'category':df['category'][i],
                'description': df['description'][i],
                'link': df['link'][i],
                'body': df['content'][i],
                'body_vector': model.encode(df['content'][i], convert_to_tensor=True).tolist(),
                'pubDate': df['pubDate'][i]
                }
            res = es.index(index="news_index", id=i, body=news)
            
time.sleep(1)
res=es.get(index='news_index',id=1)
print(res)
print('Process complete')


In [ ]:
!curl -X GET "localhost:9200/_cat/indices?v"

#Συνάρτηση για την εύρεση των ειδήσεων

In [ ]:
def find_news(user_input, search):
  if search=="elsearch": # αν έχει επιλεγεί αναζήτηση με elasticsearch
    es = Elasticsearch()
    SEARCH_SIZE = 10 # Μέγιστος αριθμός αποτελεσμάτων που θα επιστραφούν
    query_vector = model.encode(user_input, convert_to_tensor=True).tolist() # δημιουργία embeddings από το ερώτημα που υποβάλλεται
    print(query_vector[0:10])
    # χρήση του ερωτήματος
    script_query = {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'body_vector') + 1.0",
                    "params": {"query_vector": query_vector}
                }
            }
        }

        # αναζήτηση στο index
    res = es.search(
        index='news_index',  # το όνομα του index
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source": {"includes": ["title", "body", "body_vector", "link"]}
            }
        )

    # δημιουργία λίστας με τις ειδήσεις που βρέθηκαν
    news = []
    header='<h2>Matches to query: {input} </h2>'
    news.append(header.format(input=user_input))
    base_element = '<a href="{url}">{title}</a> - <b>News</b>: <i>{body}</i>'
    for hit in res['hits']['hits']:
        news.append(base_element.format(url=hit['_source']['link'], title=hit['_source']['title'],body=hit['_source']['body']))
    return news
  else:  #αν έχει επιλεγεί αναζήτηση με faiss
    t=time.time()
    query_vector = model.encode([user_input])
    k = 10
    top_k = index.search(query_vector, k)
    print(top_k)
    results=top_k[1].tolist()[0]
   
    print('totaltime: {}'.format(time.time()-t))
    # δημιουργία λίστας με τις ειδήσεις που βρέθηκαν
    news = []
    header='<h2>Matches to query: {input} </h2>'
    news.append(header.format(input=user_input))
    base_element =  '<a href="{url}">{title}</a> - <b>News</b>: <i>{body}</i>'
    for _id in top_k[1].tolist()[0]:
        news.append(base_element.format(url=df['link'][_id], title=df['title'][_id],body=df['content'][_id]))   
    
    return news


# Εκκίνηση της διεπαφής χρήστη

In [ ]:
!pip install flask-ngrok
!pip install flask==0.12.2  # Νεότερες εκδόσεις του flask δεν δουλεύουν στο  Colab
                            # Βλέπε https://github.com/plotly/dash/issues/257

!pip install pyngrok



In [ ]:
from pyngrok import ngrok
!ngrok authtoken 22f2I1Y6Jkb0FSVz8J6kgWu9GPc_y3BHnWUbv3CA8NAWQy9L # αποθήκευση του κλειδιού Authtoken στο αρχείο ρυθμίσεων ngrok.yml 
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
from google.colab import drive
# στο drive υπάρχει ο φάκελος templates με τα αρχεια index.htlm και results.html
drive.mount('/content/drive')
app = Flask(__name__,template_folder='/content/drive/My Drive/templates') 

run_with_ngrok(app)
@app.after_request
def after_request(response):
    response.headers["Cache-Control"] = "no-cache, no-store, must-revalidate, public, max-age=0"
    response.headers["Expires"] = '0'
    response.headers["Pragma"] = "no-cache"
    return response

# σελίδα Index
@app.route('/', methods=['get', 'post'])            

def index_page():
    return render_template('index.html')

# Λήψη δεδομένων από τη φόρμα στη σελίδα index, και πέρασμα αυτών στη συνάρτηση find_news()
@app.route('/results', methods=['get', 'post'])
def results_page():
    if request.method == 'POST':
        user_input = request.form["user_input"]
        print(user_input)
        check = request.form["search"]
        if check=="elsearch": # αν έχει επιλεγεί αναζήτηση με elasticsearch
           search="elsearch" 
        else:
           search="faiss"  # αν έχει επιλεγεί αναζήτηση με faiss
               
        news = find_news(user_input,search)       
        return render_template('results.html',data=news)  # επιστροφή της σελίδας results.html με τα αποτελέσματα αναζήτησης
    else:
        return "An error was encountered"
if __name__ == '__main__':
    app.run()